In [ ]:
!pip install --upgrade imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install glob2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import the necessary packages
from PIL import Image, ImageFilter
from pathlib import Path
import glob
from zipfile import ZipFile
from matplotlib import pyplot as plt
from imutils import face_utils
import numpy as np
import pandas as pd
import argparse
import imutils
import dlib
import cv2
import os
import sys



In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2


--2022-05-27 16:05:22--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  30.5MB/s    in 2.0s    

2022-05-27 16:05:24 (30.5 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
!bzip2 -d  '/content/facial-landmarks/shape_predictor_68_face_landmarks.dat.bz2'


bzip2: Can't open input file /content/facial-landmarks/shape_predictor_68_face_landmarks.dat.bz2: No such file or directory.


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/AFLW2000-3D.zip

Archive:  /content/drive/MyDrive/AFLW2000-3D.zip
   creating: AFLW2000/
   creating: AFLW2000/Code/
  inflating: AFLW2000/Code/DrawSolidHead.m  
  inflating: AFLW2000/Code/DrawTextureHead.m  
  inflating: AFLW2000/Code/main_show_without_BFM.m  
  inflating: AFLW2000/Code/main_show_with_BFM.m  
   creating: AFLW2000/Code/Mex/
  inflating: AFLW2000/Code/Mex/compile.m  
  inflating: AFLW2000/Code/Mex/Tnorm_Vnorm.h  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.cpp  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.mexw64  
   creating: AFLW2000/Code/ModelGeneration/
  inflating: AFLW2000/Code/ModelGeneration/ModelGenerate.m  
  inflating: AFLW2000/Code/ModelGeneration/model_info.mat  
  inflating: AFLW2000/Code/Model_Exp.mat  
  inflating: AFLW2000/Code/Model_Shape_Sim.mat  
  inflating: AFLW2000/Code/NormDirection.m  
  inflating: AFLW2000/Code/readme.txt  
  inflating: AFLW2000/Code/RotationMatrix.m  
  inflating: AFLW2000/Code/Tnorm_VnormC.mexw64  
  inflating: AFLW2000/image00002.jpg  
  in

In [ ]:
# with ZipFile('/content/AFLW2000_HPose.zip','r') as zipobj:
#   zipobj.extractall()


In [ ]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/facial-landmarks/facial-landmarks.zip
!unzip -qq facial-landmarks.zip
%cd facial-landmarks

--2022-05-27 16:06:55--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/facial-landmarks/facial-landmarks.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.218.178.33
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.218.178.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74572347 (71M) [binary/octet-stream]
Saving to: ‘facial-landmarks.zip’

facial-landmarks.zi 100%[===================>]  71.12M  27.6MB/s    in 2.6s    

2022-05-27 16:06:58 (27.6 MB/s) - ‘facial-landmarks.zip’ saved [74572347/74572347]

/content/facial-landmarks


In [ ]:
# predictor=dlib.shape_predictor('/content/facial-landmarks/shape_predictor_68_face_landmarks.dat')



In [ ]:
def plt_imshow(title, image):
  # convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

In [ ]:
args = {
	"shape_predictor": "shape_predictor_68_face_landmarks.dat",
	"image": "images/example_01.jpg"
}

In [ ]:
img_list_train= []

for images in glob.glob('/content/facial-landmarks/AFLW2000/*.jpg'):
   
    # check if the image ends with png
    img_list_train.append(images)

img_list_train.sort()


In [ ]:
len(img_list_train)

2000

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
points_nums = [31, 37, 40, 43, 46]
first_img = []
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

# load the input image, resize it, and convert it to grayscale
image = cv2.imread(img_list_train[0])
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)



In [ ]:
# first_img = []

# for (i, rect) in enumerate(rects):
# 	# determine the facial landmarks for the face region, then
# 	# convert the facial landmark (x, y)-coordinates to a NumPy
# 	# array
# 	shape = predictor(gray, rect)
 
# for i in points_nums:
#     first_img.append(np.array([shape.part(i).x]))
#     first_img.append(np.array([shape.part(i).y]))

# first_img_arr = np.array(first_img)
# first_img_arr
# df = pd.DataFrame(first_img_arr.T,columns = ['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5'])
# # df.insert(first_img_arr.T,)
# df
# # 0	206	328	152	161	177	198	336	182	373	216

In [ ]:

array = np.array([np.linspace(1,136, num= 136)])

# load the input image, resize it, and convert it to grayscale
for img in img_list_train :
  all_imgs = []
  image = cv2.imread(img)
  image = imutils.resize(image, width=500)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # detect faces in the grayscale image
  rects = detector(gray, 1)
  for (i, rect) in enumerate(rects):
    shape = predictor(gray, rect)
	  # shape_array = face_utils.shape_to_np(shape)
  for i in range(0,68):
    all_imgs.append(np.array([shape.part(i).x]))
    all_imgs.append(np.array([shape.part(i).y])) 
  all_imgs_arr = np.array(all_imgs)
  array = np.vstack([array,all_imgs_arr.T])



In [ ]:
array= np.delete(array, (0) , axis=0)
# columns = ['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5']
df = pd.DataFrame(array)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 136 entries, 0 to 135
dtypes: float64(136)
memory usage: 2.1 MB


In [ ]:
# # loop over the face detections
# for (i, rect) in enumerate(rects):
# 	# determine the facial landmarks for the face region, then
# 	# convert the facial landmark (x, y)-coordinates to a NumPy
# 	# array
	# shape = predictor(gray, rect)
	# shape_array = face_utils.shape_to_np(shape)
# 	print(shape.part(4))
	

# 	# convert dlib's rectangle to a OpenCV-style bounding box
# 	# [i.e., (x, y, w, h)], then draw the face bounding box
# 	# (x, y, w, h) = face_utils.rect_to_bb(rect)
# 	# cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 	# show the face number
# 	# cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
# 	# cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 	# loop over the (x, y)-coordinates for the facial landmarks
# 	# and draw them on the image
# 	x_y_list = []
# 	for (x, y) in shape_array:
# 		cv2.circle(image, (x, y), 3, (0, 0, 255), -1)
# # show the output image with the face detections + facial landmarks
# print(shape_array)

# plt_imshow("Output", image)

In [ ]:
dff = pd.read_csv('/content/angle_data.csv')
dff_copy = dff.drop(columns=['Unnamed: 0', 'img_name'])
dff_copy.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pitch   2000 non-null   float64
 1   yaw     2000 non-null   float64
 2   roll    2000 non-null   float64
dtypes: float64(3)
memory usage: 47.0 KB


In [ ]:
df_copy = df.copy()
df_copy['yaw'] = dff_copy['yaw']
df_copy['pitch'] = dff_copy['pitch']
df_copy['roll'] = dff_copy['roll']
df_copy.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,yaw,pitch,roll
0,140.0,196.0,140.0,227.0,144.0,258.0,149.0,289.0,156.0,321.0,...,349.0,252.0,361.0,238.0,362.0,225.0,357.0,1.044306,-22.874239,4.908885
1,209.0,249.0,204.0,264.0,201.0,282.0,202.0,301.0,205.0,320.0,...,339.0,231.0,339.0,226.0,338.0,223.0,338.0,68.155235,26.932741,17.243670
2,195.0,288.0,195.0,305.0,198.0,323.0,205.0,341.0,214.0,358.0,...,340.0,245.0,355.0,240.0,356.0,234.0,355.0,50.485413,-10.579652,-13.570644
3,169.0,279.0,176.0,302.0,186.0,323.0,201.0,344.0,218.0,363.0,...,336.0,270.0,353.0,261.0,356.0,252.0,356.0,17.143373,-10.048456,-21.392782
4,169.0,279.0,176.0,302.0,186.0,323.0,201.0,344.0,218.0,363.0,...,336.0,270.0,353.0,261.0,356.0,252.0,356.0,68.640549,-50.544579,-59.207973


In [ ]:
X = np.array(df)
y = np.array(dff_copy)


# Machine Learning Modeling

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression , Lasso , Ridge
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score , f1_score, r2_score 
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor






In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
model = MultiOutputRegressor(DecisionTreeRegressor(criterion = 'squared_error'))
model.fit(X_train, y_train)


MultiOutputRegressor(estimator=DecisionTreeRegressor())

In [ ]:
model.score(X_train,y_train)


0.5624593563136909

In [ ]:
y_pre = model.predict(X_train)
r2_score(y_train,y_pre)

0.5624593563136909

In [ ]:
y_train_df = pd.DataFrame(y_train)
y_train_df[0]

0      -12.484629
1      -17.030193
2      -25.357058
3      -16.137459
4      -71.447075
          ...    
1595   -23.988100
1596    20.870047
1597    -6.763399
1598    -0.674434
1599    18.787455
Name: 0, Length: 1600, dtype: float64

In [ ]:
y_test_df = pd.DataFrame(y_test)
y_test_df[0]

0     -21.789831
1      -0.075722
2       7.048895
3      -8.241077
4     -29.447075
         ...    
395    21.947229
396    -4.300839
397   -14.805589
398   -11.297882
399    19.150320
Name: 0, Length: 400, dtype: float64

In [ ]:
tree = DecisionTreeRegressor(criterion = 'squared_error')
tree.fit(X_train,y_train_df[0])
tree_predict = tree.predict(X_test)
# tree_predict
# accuracy_score(np.array(y_test_df[0]),tree_predict)
tree.score(X_test,y_test_df[2])

-0.5310938681741364